# Assignment - 2
### @Author: Honi, Selahaddin (honis@kaist.ac.kr)
### @Date: 2022 April

Program requires OpenCV for image I&O and matrix computations for NumPy. `Noise Addition` section contains Gaussian and S&P noise addition functions to apply these onto `lena` and `cameraman` images. Noisy images are stored in `A,B,C` lists. `Mean and Order-statistic Filters` functions are called inside a nested-loop for each neighborhood of each noisy images in `A,B,C` lists. Parameters (if exist) are determined in function arguments by default. `Adaptive Filters` have a bit different loop design such that `Local Noise Reduction Filter` calls `estimateNoiseVar()` function, firstly, to predict global image noise variance by calculating image histogram. `AdaptiveMedian()` filter function gets each neighborhood in the image and executes recursively. 

Neighborhood size `k=3` for all filters; however, adaptive median filter can change the neighborhood size if specified conditions are not satisfied. Therefore, maximum allowed neighborhood size is limited by `Smax=30`.

## Noise Addition

In [1]:
import cv2 as cv
import numpy as np

def add_gaussian_noise(img,sigma):
    gauss = np.random.normal(0,sigma,img.shape[:2])
    img_noised = img + gauss
    return img_noised

def add_salt_and_pepper_noise(img,prob):
    img_noised = img.copy()
    probs = np.random.random(img.shape[:2])
    img_noised[probs <     (prob / 2)] = 0
    img_noised[probs > 1 - (prob / 2)] = 255
    return img_noised   

In [2]:
path = './img/'
fnames = ['lena','cm']

sigma = 20  # Gaussian noise
prob = 0.25 # S&P noise

O = [] # original images
A = [] # S&P images
B = [] # G images 
C = [] # S&P+G images

for fname in fnames:
    img = cv.imread(path+fname+'.png',0)

    img_sp = add_salt_and_pepper_noise(img,prob=prob)
    img_g = add_gaussian_noise(img,sigma=sigma)
    img_spg = add_gaussian_noise(img_sp, sigma=sigma)

    cv.imwrite(path+fname+'_sp'+'.png',img_sp)
    cv.imwrite(path+fname+'_g'+'.png',img_g)
    cv.imwrite(path+fname+'_spg'+'.png',img_spg)

    O.append([img,fname])
    A.append([img_sp,fname+'_sp'])
    B.append([img_g,fname+'_g'])
    C.append([img_spg,fname+'_spg'])

## Implementation of Mean and Order-statistic Filters

In [3]:
# Mean Filters 
# ================================================

def arithmetic(neighborhood):
    m,n = neighborhood.shape
    val = 1/(m*n) * np.sum(neighborhood)
    return val

def geometric(neighborhood):
    m,n = neighborhood.shape
    prod = 1
    for elem in neighborhood.flatten():
        prod *= (elem + 1e-1)
    val = prod ** (1/(m*n))
    return val

def harmonic(neighborhood):
    m,n = neighborhood.shape
    den = 1e-10 # prevent divide by zero 
    for elem in neighborhood.flatten():
        den += 1/(elem+1e-2) # prevent divide by zero 
    val = (m*n) / den 
    return val

def contraharmonic(neighborhood,Q=2):
    nom = 0
    den = 1e-10
    for elem in neighborhood.flatten():
        nom += elem**(Q+1)
        den += elem**Q
    val = nom/den
    return val

# Order-statistic Filters
# ================================================

def median(neighborhood):
    m,n = neighborhood.shape
    mid = m*n//2
    sort = np.sort(neighborhood.flatten())
    val = sort[mid]
    return val

def max(neighborhood):
    val = neighborhood.max()
    return val

def min(neighborhood):
    val = neighborhood.min()
    return val

def midpoint(neighborhood):
    val = 0.5 * (neighborhood.max()+neighborhood.min())
    return val

def alpha_trimmed(neighborhood,d=2):
    m,n = neighborhood.shape
    sort = np.sort(neighborhood.flatten())
    trim = sort[d//2:-d//2]
    val = 1/(m*n-d) * np.sum(trim)
    return val

In [ ]:
k = 3 # neigborhood size 

filters_mean = [arithmetic,geometric,harmonic,contraharmonic]
filters_order_stat = [median, max, min, midpoint, alpha_trimmed]

for imgs in [A,B,C]:
    for img,fname in imgs:

        out = img.copy()
        H,W = img.shape

        for applyFilter in filters_mean + filters_order_stat:

            for y in range(H):
                for x in range(W):

                    # check neighborhood inside the image
                    if y < k//2 or y >= H-(k//2) \
                        or x < k//2 or x >= W-(k//2):
                        continue
                    
                    # y,x center point location
                    # crop neighborhood area centered on (x,y) from original image
                    neighborhood = img[y-(k//2):y+(k//2)+1, x-(k//2):x+(k//2)+1]
                    # pass this neigborhood to specified filter
                    out[y,x] = applyFilter(neighborhood)

            out_path = path+fname+'_'+applyFilter.__name__+'_'+str(k)+'.png'
            cv.imwrite(out_path,out)
            print(fname+'_'+applyFilter.__name__)

## Adaptive, Local Noise Reduction Filter 

In [5]:
def estimateNoiseVar(img,L=256):
    # Use histogram to estimate noise variance
    levels = np.arange(L)
    p = []
    m = 0

    img1d = img.flatten()
    pixel_total = len(img1d)

    for r in levels:
        idx = np.where(img1d==r)[0]
        count = idx.shape[0]
        p_of_r = count/pixel_total
        p.append(p_of_r)
        m += r*p_of_r

    var_noise = 0
    for r in range(L):
        var_noise += ((levels[r]-m)**2) * p[r]
    
    return var_noise

In [6]:
k = 3 # neigborhood size 

for imgs in [A,B,C]:
    for img,fname in imgs:

        out = img.copy()
        H,W = img.shape

        # global image noise variance
        var_noise = estimateNoiseVar(img)

        for y in range(H):
            for x in range(W):

                # check neighborhood inside the image
                if y < k//2 or y >= H-(k//2) \
                    or x < k//2 or x >= W-(k//2):
                    continue
                
                # y,x center point location
                # crop neighborhood area centered on (x,y) from original image
                neighborhood = img[y-(k//2):y+(k//2)+1, x-(k//2):x+(k//2)+1]
                
                # local neighbothood variance 
                var_local = np.var(neighborhood.flatten())
                # variance ratio check
                ratio = 1 if var_noise>var_local else var_noise/var_local
                # neighborhood intensity average
                z_bar = np.mean(neighborhood.flatten())
                # center intensity 
                g_of_xy = img[y,x]
                
                # adaptive restoration
                out[y,x] = g_of_xy - ratio * (g_of_xy - z_bar)

        out_path = path+fname+'_adaptive_reduction_'+str(k)+'.png'
        cv.imwrite(out_path,out)

## Adaptive, Median Filter

In [7]:
def AdaptiveMedian(z_ret,img,k,x,y,H,W,Smax=30):

    # check neighborhood inside the image
    if y < k//2 or y >= H-(k//2) \
        or x < k//2 or x >= W-(k//2):
        return z_ret

    # y,x center point location
    # crop neighborhood area centered on (x,y) from original image
    neighborhood = img[y-(k//2):y+(k//2)+1, x-(k//2):x+(k//2)+1]
    
    # min-max-median intensities
    z_min = np.min(neighborhood)
    z_max = np.max(neighborhood)
    z_med = np.median(neighborhood)

    # center intensity
    z_xy = img[y,x]

    # return intensity := z_med (by default)
    z_ret = z_med

    # Level-B
    if z_min < z_med and z_med < z_max:
        if z_min < z_xy and z_xy < z_max:
            z_ret = z_xy
        else:
            z_ret = z_med
    # Level-A
    else:
        k += 2 # increase neighborhood size 
        if k<=Smax:
            z_ret = AdaptiveMedian(z_ret,img,k,x,y,H,W,Smax=Smax)
        else:
            z_ret = z_med

    return z_ret

In [8]:
k = 3 # neigborhood size 

for imgs in [A,B,C]:
    for img,fname in imgs:

        out = img.copy()
        H,W = img.shape

        for y in range(H):
            for x in range(W):              

                out[y,x] = AdaptiveMedian(1,img,k,x,y,H,W)

        out_path = path+fname+'_adaptive_median_'+str(k)+'.png'
        cv.imwrite(out_path,out)